## WHT ISIS Long-slit Spectroscopy Reduction

The following reduction code works directly from the full set of images in a given nights observations. As observations are added during the evening, the ImageFileCollection object defined at the beginning needs to be refreshed.

Files can be rsync'd from the whtobs computer using the following command:

`$ rsync -av whtobs@taurus.ing.iac.es:/obsdata/whta/20170624/ 20170624/`

where 20170624/ is set to the relevant observation night.
The starting point for this code was the very helpful scripts of Steve Crawford. However, various changes have been made and key steps (e.g. wavelength calibration/flat processing) differ.

In [ ]:
import numpy as np
from multiprocessing import Pool
from functools import partial
import os

import astropy.units as u
from astropy.io import fits
from astropy.modeling import models, fitting
from astropy.stats import sigma_clip, mad_std
from scipy.ndimage import binary_dilation
from astropy.utils.console import ProgressBar
import ccdproc
from ccdproc import ImageFileCollection, CCDData
from astropy.stats import sigma_clipped_stats
import scipy.optimize as optimize



import scipy
import matplotlib.pyplot as plt

def fit_chebyshev(row, degree=7, grow=3):
    """
    Fit Chebyshev1D model to a CCD row, including masking of outlier pixels
    
    Params
    ------
    row : array,
        Input CCD row to fit polynomial to.
    degree : int,
        Chebyshev Polynomial Order
    grow : int,
        Number of iterations to dilate around masked pixels
    """

    fitter = fitting.LinearLSQFitter()
    input_mask = row.mask
    clipped = sigma_clip(row, stdfunc=mad_std)
    clipped_pixels = np.array(clipped.mask+row.mask).astype('float')
    clipped_pixels = binary_dilation(clipped_pixels, iterations=grow)

    row[clipped_pixels==1] = np.median(row)
    masked_row = np.ma.array(data=row, 
                             mask=(clipped_pixels == 1), 
                             fill_value=np.median(row))
    x = np.arange(len(row))
    model = models.Chebyshev1D(degree=degree)
    fitted_model = fitter(model, x, row)
    return fitted_model(np.arange(len(row)))


def fit_background(data, degree=7, grow=3, verbose=True, njobs=2):
    """
    Parallelised background estimation for longslit CCD image
    
    Params
    ------
    data : array,
        Input CCD data for background estimation.
    degree : int,
        Chebyshev Polynomial Order
    grow : int,
        Number of iterations to dilate around masked pixels
    njobs : int
        Number of processes to initiate for fitting
    """
    kwargs={'degree': degree, 'grow': grow}    
    p = Pool(njobs)
    fitted_sky = p.map(partial(fit_chebyshev, **kwargs), data)
    p.close()
    return np.array(fitted_sky).astype('float')


In [ ]:
obs_night = "20170626"

### Directory with all the raw files from an observing night
ic1 = ImageFileCollection("/Users/aayushsaxena/Desktop/phd/HIZESP/WHTRUN/%s/" %obs_night)

try:
    os.mkdir("/Users/aayushsaxena/Desktop/phd/HIZESP/WHTRUN/%s/processed" %obs_night)
except OSError:
    pass


In [ ]:
### show files to make sure path is correct

print(ic1.files)

print(ic1.files_filtered(obstype='Bias', isiarm='Blue arm'))

## Read or Create the bias frames

In [ ]:
### Run this the first time to create master bias frames
### Create blue master bias
blue_bias_list = []

for filename in ic1.files_filtered(obstype='Bias', isiarm='Blue arm'):
    print ic1.location + filename
    ccd = CCDData.read(ic1.location + filename, unit = u.adu)
    #ccd = ccdproc.create_deviation(ccd, gain=ccd.header['GAIN']*u.electron/u.adu, 
     #                              readnoise=ccd.header['READNOIS']*u.electron)
    #this has to be fixed as the bias section does not include the whole section that will be trimmed
    ccd = ccdproc.subtract_overscan(ccd, median=True,  overscan_axis=0, fits_section='[1:966,4105:4190]')
    ccd = ccdproc.trim_image(ccd, fits_section=ccd.header['TRIMSEC'])
    blue_bias_list.append(ccd)
    
master_bias_blue = ccdproc.combine(blue_bias_list, method='median')
master_bias_blue.write('../%s/processed/master_bias_blue.fits' %obs_night, clobber=True)

### For future reductions, simply read in the master bias file: uncomment this line and comment out 
### the block of code above.
# master_bias_blue = CCDData.read('/Users/aayushsaxena/Desktop/phd/HIZESP/WHTRUN/%s/processed/master_bias_blue.fits' %obs_night, unit=u.adu)


### Create red master bias
red_bias_list = []

for filename in ic1.files_filtered(obstype='Bias', isiarm='Red arm'):
    print ic1.location + filename
    ccd = CCDData.read(ic1.location + filename, unit = u.adu)
    #ccd = ccdproc.create_deviation(ccd, gain=ccd.header['GAIN']*u.electron/u.adu, 
    #                               readnoise=ccd.header['READNOIS']*u.electron)
    #this has to be fixed as the bias section does not include the whole section that will be trimmed
    ccd = ccdproc.subtract_overscan(ccd, median=True,  overscan_axis=0, fits_section='[1:966,4105:4190]')
    ccd = ccdproc.trim_image(ccd, fits_section=ccd.header['TRIMSEC'] )
    red_bias_list.append(ccd)

master_bias_red = ccdproc.combine(red_bias_list, method='median')
master_bias_red.write('../%s/processed/master_bias_red.fits' %obs_night, clobber=True)

### uncomment the following line once master bias created (see above)
# master_bias_red = CCDData.read('/Users/aayushsaxena/Desktop/phd/HIZESP/WHTRUN/%s/processed/master_bias_red.fits' %obs_night, unit=u.adu)


print("Master Biases created")

## Create master flats

In [ ]:
blue_flat_list = []
for filename in ic1.files_filtered(obstype='Flat', isiarm='Blue arm', object='well good flat b'):
    ccd = CCDData.read(ic1.location + filename, unit = u.adu)
    #ccd = ccdproc.create_deviation(ccd, gain=ccd.header['GAIN']*u.electron/u.adu, 
     #                              readnoise=ccd.header['READNOIS']*u.electron)
    #this has to be fixed as the bias section does not include the whole section that will be trimmed
    ccd = ccdproc.subtract_overscan(ccd, median=True,  overscan_axis=0, fits_section='[1:966,4105:4190]')
    ccd = ccdproc.trim_image(ccd, fits_section=ccd.header['TRIMSEC'] )
    ccd = ccdproc.subtract_bias(ccd, master_bias_blue)
    blue_flat_list.append(ccd)

master_flat_blue = ccdproc.combine(blue_flat_list, method='median')
# convolved_flat_blue = convolve(master_flat_blue.data, kernel, boundary='extend')

master_flat_blue.write('../%s/processed/master_flat_blue.fits' %obs_night, clobber=True)

# master_flat_blue.data /= convolved_flat_blue
# master_flat_blue.write('../%s/processed/master_flat_norm_blue.fits' %obs_night, clobber=True)

### Uncomment the following line once flats have been created (just like biases)
# master_flat_blue = CCDData.read('/Users/aayushsaxena/Desktop/phd/HIZESP/WHTRUN/%s/processed/master_flat_norm_blue.fits' %obs_night)

print("Blue Flat created")

In [ ]:
from astropy.convolution import convolve, Gaussian2DKernel

kernel = Gaussian2DKernel(25)

red_flat_list = []
for filename in ic1.files_filtered(obstype='Flat', isiarm='Red arm', object='well good flat r'):
    ccd = CCDData.read(ic1.location + filename, unit = u.adu)
    #ccd = ccdproc.create_deviation(ccd, gain=ccd.header['GAIN']*u.electron/u.adu, 
     #                              readnoise=ccd.header['READNOIS']*u.electron)
    #this has to be fixed as the bias section does not include the whole section that will be trimmed
    ccd = ccdproc.subtract_overscan(ccd, median=True,  overscan_axis=0, fits_section='[1:966,4105:4190]')
    ccd = ccdproc.trim_image(ccd, fits_section=ccd.header['TRIMSEC'] )
    ccd = ccdproc.subtract_bias(ccd, master_bias_red)    
    red_flat_list.append(ccd)

master_flat_red = ccdproc.combine(red_flat_list, method='median')
# convolved_flat_red = convolve(master_flat_red.data, kernel, boundary='extend')

master_flat_red.write('../%s/processed/master_flat_red.fits' %obs_night, clobber=True)

# master_flat_red.data /= convolved_flat_red
# master_flat_red.write('../%s/processed/master_flat_norm_red.fits' %obs_night, clobber=True)

### Uncomment the following line once master flats have been created
# master_flat_red = CCDData.read('/Users/aayushsaxena/Desktop/phd/HIZESP/WHTRUN/%s/processed/master_flat_norm_red.fits' %obs_night)

print("Red Flat fields created")

## Reduce the object frames

In [ ]:
objects = ['GRG3'] # Enter object name (see from observing log)

for objname in objects:
    print(objname)
    """
    Blue Arm
    """
    blue_target_list = []
    for ifx, filename in enumerate(ic1.files_filtered(obstype='TARGET', isiarm='Blue arm', object=objname)):
        print('{0} {1}'.format(ifx+1, filename))
        hdu = fits.open(ic1.location + filename)
        ccd = CCDData(hdu[1].data, header=hdu[0].header+hdu[1].header, unit = u.adu)
        #this has to be fixed as the bias section does not include the whole section that will be trimmed

        ccd = ccdproc.cosmicray_lacosmic(ccd, sigclip=4., niter=10, sigfrac=0.3, psffwhm=2.5, 
                                         gain=ccd.header['GAIN'], readnoise=ccd.header['READNOIS'])
        ccd = ccdproc.subtract_overscan(ccd, median=True,  overscan_axis=0, fits_section='[1:966,4105:4190]')
        ccd = ccdproc.trim_image(ccd, fits_section=ccd.header['TRIMSEC'] )
        ccd = ccdproc.subtract_bias(ccd, master_bias_blue)
        ccd = ccdproc.flat_correct(ccd, master_flat_blue)
        
        
        # Do sky subtraction
        ccd.mask[:,786:800] = True
        sky = fit_background(np.ma.array(ccd.data, mask=ccd.mask))
        ccd.data -= sky
        
        # Rotate Frame
        ccd.data = ccd.data.T
        ccd.mask = ccd.mask.T
        blue_target_list.append(ccd)
        #ccd.write('obj_'+filename, clobber=True)

    blue_target = ccdproc.combine(blue_target_list, method='average')
    blue_target.write('/Users/aayushsaxena/Desktop/phd/HIZESP/WHTRUN/%s/processed/{0}_blue.fits'.format(blue_target_list[0].header['object']) %obs_night, clobber=True)
    
print("Reduced Blue")

In [ ]:
### Reduce Red

objects = ['GRG3'] # Enter the object name (see from observing log)

for objname in objects:
    print(objname)
    """
    Red Arm
    """
    red_target_list = []
    for ifx, filename in enumerate(ic1.files_filtered(obstype='TARGET', isiarm='Red arm', object=objname)):
        print('{0} {1}'.format(ifx+1, filename))
        hdu = fits.open(ic1.location + filename)
        ccd = CCDData(hdu[1].data, header=hdu[0].header+hdu[1].header, unit = u.adu)
        #this has to be fixed as the bias section does not include the whole section that will be trimmed

        ccd = ccdproc.cosmicray_lacosmic(ccd, sigclip=4., niter=10, sigfrac=0.3, psffwhm=2.5, 
                                         gain=ccd.header['GAIN'], readnoise=ccd.header['READNOIS'])
        ccd = ccdproc.subtract_overscan(ccd, median=True,  overscan_axis=0, fits_section='[1:966,4105:4190]')
        ccd = ccdproc.trim_image(ccd, fits_section=ccd.header['TRIMSEC'] )
        ccd = ccdproc.subtract_bias(ccd, master_bias_red)
        ccd = ccdproc.flat_correct(ccd, master_flat_red)
        
        
        # Do sky subtraction
#         ccd.mask[:,690:700] = True
#         sky = fit_background(np.ma.array(ccd.data, mask=ccd.mask))
#         ccd.data -= sky
        
        # Rotate Frame
        
        ccd.data = ccd.data.T
        ccd.mask = ccd.mask.T
        #ccd.write('obj_'+filename, clobber=True)
        red_target_list.append(ccd)

    red_target = ccdproc.combine(red_target_list, method='average')
    # red_target.write('{0}_red.fits'.format(red_target_list[0].header['object']), clobber=True)

    red_target.mask[785:800,:] = True
    red_sky = fit_background(np.ma.array(red_target.data.T, mask=red_target.mask.T)).T
    red_target.data -= red_sky
    
    red_target.write('/Users/aayushsaxena/Desktop/phd/HIZESP/WHTRUN/%s/processed/{0}_red.fits'.format(red_target_list[0].header['object']) %obs_night, clobber=True)
    
print("Reduced Red")

### Currently in the process of optimising wavelength and flux calibration. 

The current routines could definitely be improved a lot. I'll update them the notebook as I make the calibration process better. The calibration is currently done on the extracted 1D spectrum, but a more efficient way would be to wavelength calibrate the 2D spectrum to also visualise the spatial distribution of line emission